Some code in this notebook has been adapted from the work of Zhongy et al. and Greydanus et al. and from the report and code of Jonas Perolini.

Their code is available in the following repositories :[
Symplectic-ODENet](https://github.com/Physics-aware-AI/Symplectic-ODENet) and [hamiltonian-nn](https://github.com/greydanus/hamiltonian-nn)

# Imports & Setting up directories

In [ ]:
try:
    import google.colab

    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    PATH = "./"  # './drive/MyDrive/1_SP_Ham_func/'
    from google.colab import drive

    drive.mount("/content/drive")

    %cd /content/drive/MyDrive/1_SP_Ham_func/furuta_pendulum/
    %pip install torchdiffeq
    from src.data import *
    from src.dynamics import *
    from src.models import *
    from src.train import *
    from src.plots import *
    from src.trajectories import *
    from src.utils import *
    from src.autoencoder_train import *
    from src.autoencoder_plots import *
else:
    import sys

    sys.path.insert(0, "..")
    import os

    PATH = "../"
    from src.data import *
    from src.dynamics import *
    from src.models import *
    from src.train import *
    from src.plots import *
    from src.trajectories import *
    from src.utils import *
    from src.autoencoder_train import *
    from src.autoencoder_plots import *

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader, random_split
import torch

from torch.optim.lr_scheduler import LinearLR

from torchdiffeq import odeint_adjoint as odeint_adjoint

# func must be a nn.Module when using the adjoint method
from torchdiffeq import odeint as odeint

import time as time
import json
import os

# setting seeds
import random

In [ ]:
%load_ext autoreload
%autoreload 2

#### AE alone

In [ ]:
set_all_seeds(manualSeed=123, new_results=False)
device = set_device()  # set it to gpu if it is available

# Parameters to generate the dataset
furuta_type = "fake"  # 'real' or 'fake'
Ts, noise_std, C_q1, C_q2, g, Jr, Lr, Mp, Lp = set_furuta_params(which=furuta_type)
utype = None  # 'chirp' or 'sine' or 'tanh' or 'multisine' or 'step' or None
u_func = U_FUNC(utype=utype)  # instantiate the input function u(t)
u_func.params["T"] = 1.5
u_func.params["f0"] = 0
u_func.params["f1"] = 2  # 4 # 1.4
u_func.params["scale"] = 0.0001  # for fake : 0.5 or 0.1 for real : 0.0001
gtype = None  # 'simple' or None
# instantiate the input function G(q,p) (here it is constant)
g_func = G_FUNC(gtype=gtype)
init_method = "random_nozero"  # 'random_nozero' # 'random_closetopi'
time_steps = 800  # length of a trajectory
num_trajectories = 125  # number of trajectories in total
proportion = 0.8  # train test proportion
batch_size = 100  # batch size used by dataloader
w_rescale = [1, 1000, 1, 10000]  # [1, 1000, 1, 10000]  # [1, 9000, 1, 10000]
shuffle = False  # shuffle sample in the batches between epochs
# 'hamiltonian' or 'newtonian', newtonian if you want [q1,q1_dot,q2,q2_dot]
coord_type = "newtonian"
min_max_rescale = False  # rescale the training trajectories
# which dimensions to rescale if using min_max_rescale, so that nothing is divided by zero
rescale_dims = [1, 1, 1, 1]

# Parameters for the training procedure
resnet_config = None
alternating = False  # for Input_HNN, if G is a neural network, train
horizon_type = "auto"  # 'auto' or 'constant'
horizon = False  # if horizon_type == 'constant', use this horizon
loss_type = "L2"  # 'L2' or 'L2weighted'
collect_grads = False  # collect gradients in all layers at every epoch
# rescale the difference between nominal and train by the min max of train trajectory in loss function
rescale_loss = False
grad_clip = True  # activate gradient clipping
lr_schedule = False  # activate lr schedule
begin_decay = 600  # epoch at which lr starts decaying
weights = [1.0, 1.0, 1.0, 1.0]  # weights for the loss functions

horizon_list = [300]
switch_steps = [500]
epoch_number = sum(switch_steps)  # total number of training epochs

In [ ]:
train_loader, test_loader = load_data_device(
    device,
    init_method,
    w_rescale,
    u_func,
    g_func,
    time_steps,
    shuffle=shuffle,
    num_trajectories=num_trajectories,
    coord_type=coord_type,
    proportion=proportion,
    batch_size=batch_size,
    Ts=Ts,
    noise_std=noise_std,
    C_q1=C_q1,
    C_q2=C_q2,
    g=g,
    Jr=Jr,
    Lr=Lr,
    Mp=Mp,
    Lp=Lp,
    min_max_rescale=min_max_rescale,
    rescale_dims=rescale_dims,
)

In [ ]:
autoencoder = Autoencoder(nb_hidden_layers=1,  hidden_dim=90, activation='tanh', config = 'latent') # 'x+sin(x)^2'
autoencoder.to(device)
count_parameters(autoencoder)

In [ ]:
H_net = MLP(input_dim=4, hidden_dim=90, nb_hidden_layers=4, output_dim=1, activation="x+sin(x)^2")
model = simple_HNN(input_dim=4, H_net=H_net, device=None)
model.to(device)

num_params = count_parameters(model)
print(num_params)

In [ ]:
stats_2 = train_only_ae(autoencoder, 
                        model, 
                        device,
                        train_loader, 
                        test_loader, 
                        epochs= 1000, # 3000
                        horizon=300, 
                        lr = 1e-3,
                        w = torch.tensor([10.0,  1.0, 100.0, 1.0],device=device))

### plots

In [ ]:
plot_distribution(train_loader, save=False, path=PATH+'data/ae_distributions.png')

#### AE and Latent_ODE_HNN

In [ ]:
# model = load_model()
# count_parameters(model)

# autoencoder = Autoencoder(nb_hidden_layers=1,  hidden_dim=64, activation='tanh', config = 'encoded') # 'x+sin(x)^2'
# autoencoder.to(device)
# count_parameters(autoencoder)

In [ ]:
horizon_list = [20,50,70,90,110,130,150,170,190,210,230,250,270,290,300]
switch_steps = [200,500,500,200,200,200,200,300,300,200,200,200,200,200,200]
epoch_number = sum(switch_steps)
print(epoch_number)

In [ ]:
stats =  train_ae(model,
                  device,
                  autoencoder, 
                  train_loader, 
                  test_loader, 
                  Ts, 
                  horizon = False, 
                  horizon_type = 'auto',
                  horizon_list = horizon_list, 
                  switch_steps = switch_steps,
                  steps_ae = 0,
                  epoch_number = epoch_number,
                  w=torch.tensor([10.0,  1.0, 100.0, 1.0], device=device))

In [ ]:
## saving stats to txt file
import json

PATH = './drive/MyDrive/1_SP_Ham_func/'
stats_path = PATH+'stats/'+save_prefix+'stats.txt'

def save_stats(PATH, stats, stats_path):
    with open(stats_path, 'w') as file:
        file.write(json.dumps(stats)) # use `json.loads` to do the reverse  
    return

In [ ]:
# save model to disk

save_prefix = 'UODEHNN_usedwithae'
model_name = 'models/'+save_prefix+'model_test'
torch.save(model.state_dict(), PATH+model_name)

In [ ]:

save_prefix = 'UODEHNN_usedwithae'
model_name = 'models/'+save_prefix+'model_test'
model = load_model(hidden_dim=90, nb_hidden_layers=4)
model.load_state_dict(torch.load(PATH+model_name))
model.eval()

In [ ]:
# save AE model to disk

save_prefix = 'AE'
model_name = 'models/'+save_prefix+'model_test'
torch.save(autoencoder.state_dict(), PATH+model_name)

In [ ]:
save_prefix = 'AE'
model_name = 'models/'+save_prefix+'model_test'
autoencoder = Autoencoder(nb_hidden_layers=1,  hidden_dim=90, activation='tanh', config = 'latent') # 'x+sin(x)^2'
autoencoder.to(device)

autoencoder.load_state_dict(torch.load(PATH+model_name))
autoencoder.eval()

In [ ]:
save_stats(PATH+'aeonly', stats_2, stats_path)

In [ ]:
save_stats(PATH+'aeandHNNstats', stats, stats_path)

In [ ]:
torch.save(train_loader, PATH + 'train_loader.pt')

In [ ]:
train_loader = torch.load(PATH + 'train_loader.pt')

##### plots

In [ ]:
train_steps=290
n=70
max_timestep= 800
save_prefix = 'AUTOENCODER_MODEL_nozoom_n70_twoplots'

In [ ]:
plot_furuta_ae_twoplots(model, 
                        autoencoder, 
                        train_loader,
                        max_timestep,
                        n,
                        train_steps, time_steps, device,
                        Ts, C_q1, C_q2, g, Jr, Lr, Mp, Lp,
                        title = 'Trajectory of the generalized coordinates', 
                        file_path = PATH+'data/'+save_prefix+'TRAJECTORIES_test_set')